<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import normalize, MinMaxScaler
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full/
proteome=np.load("proteome_short.npy")

/content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full


In [4]:
scaler=MinMaxScaler()
X_train_val, X_test=train_test_split(proteome)
X_train, X_valid=train_test_split(X_train_val)

# Por el momento voy a tratar de escalar manualmente, para evitar problemas de redondeo en la
# conversión inversa

# X_train_n=scaler.fit_transform(X_train)
# X_valid_n=scaler.transform(X_valid)
# X_test_n=scaler.transform(X_test)


X_train_n=X_train/25 # el valor maximo del 'codigo numerico'
X_valid_n=X_valid/25
X_test_n=X_test/25

In [5]:
# Reshapeo para que funcione en el AE
X_train_bis=X_train_n.reshape(-1,2000,1)
X_valid_bis=X_valid_n.reshape(-1,2000,1)
X_test_bis=X_test_n.reshape(-1,2000,1)

In [8]:
X_test_bis=X_test_n.reshape(-1,2000,1)

In [111]:
tf.keras.backend.clear_session()

conv_encoder = keras.models.Sequential([
keras.layers.Reshape([2000, 1], input_shape=[2000,1]),
keras.layers.Conv1D(16, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2)
])


conv_decoder = keras.models.Sequential([
keras.layers.Conv1DTranspose(32, kernel_size=3, strides=2, padding="same",
activation="selu",
input_shape=[250, 64]),
keras.layers.Conv1DTranspose(16, kernel_size=3, strides=2, padding="same",
activation="selu"),
keras.layers.Conv1DTranspose(1, kernel_size=3, strides=2, padding="same",
activation="sigmoid"),

])


conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 250, 64)           7840      
_________________________________________________________________
sequential_1 (Sequential)    (None, 2000, 1)           7777      
Total params: 15,617
Trainable params: 15,617
Non-trainable params: 0
_________________________________________________________________


In [112]:
conv_ae.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=1.5))


In [99]:
history = conv_ae.fit(X_train_bis, X_train_bis, epochs=10,
validation_data=[X_valid_bis, X_valid_bis])

Epoch 1/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0804 - val_loss: 0.0000e+00
Epoch 2/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0804 - val_loss: 0.0000e+00
Epoch 3/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0803 - val_loss: 0.0000e+00
Epoch 4/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0803 - val_loss: 0.0000e+00
Epoch 5/10
1758/1758 [==============================] - 18s 10ms/step - loss: 0.0803 - val_loss: 0.0000e+00
Epoch 6/10
1758/1758 [==============================] - 18s 10ms/step - loss: 0.0802 - val_loss: 0.0000e+00
Epoch 7/10
 824/1758 [=============>................] - ETA: 9s - loss: 0.0803

KeyboardInterrupt: ignored

In [100]:
predictions=conv_ae.predict(X_test_bis)
# pred_num=scaler.inverse_transform(predictions.reshape(-1,2000))
# pred_num_rounded=np.vectorize(round)(pred_num)

In [96]:
def unpad(query, target):
  try:
    idx=np.where(query==0)[0][-1] + 1
  except IndexError:
    idx=0
  return query[idx:], target[idx:]

In [ ]:
rpts=5
means=np.zeros(rpts)
sums=np.zeros(rpts)
desvs=np.zeros(rpts)
for i in range(rpts):
  history = conv_ae.fit(X_train_bis, X_train_bis, epochs=10,
  validation_data=[X_valid_bis, X_valid_bis])
  scores=np.zeros(X_test_bis.shape[0])
  predictions=conv_ae.predict(X_test_bis)
  for n,(query, target) in enumerate(zip(X_test_bis.reshape(-1,2000), predictions.reshape(-1,2000))):
    qry,tgt=unpad(query,target)
    scores[n]=sum(qry-tgt)
  means[i]=scores.mean()
  desvs[i]=scores.std()
  sums[i]=scores.sum()

Epoch 1/10
1758/1758 [==============================] - 21s 11ms/step - loss: 0.0998 - val_loss: 0.0000e+00
Epoch 2/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0859 - val_loss: 0.0000e+00
Epoch 3/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0838 - val_loss: 0.0000e+00
Epoch 4/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0828 - val_loss: 0.0000e+00
Epoch 5/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0822 - val_loss: 0.0000e+00
Epoch 6/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0819 - val_loss: 0.0000e+00
Epoch 7/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0816 - val_loss: 0.0000e+00
Epoch 8/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0814 - val_loss: 0.0000e+00
Epoch 9/10
1758/1758 [==============================] - 18s 11ms/step - loss: 0.0812 - val_loss: 0.0000e+00
Epoch 10/10
1758/1758 [=====